# Classification 1. Flat approach

In [ ]:
!pip install transformers

In [3]:
from transformers import BertTokenizer
import torch
import numpy as np
from torch import nn
from transformers import BertModel
from torch.optim import Adam
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import confusion_matrix

In [ ]:
!pip install gdown

In [ ]:
!gdown https://drive.google.com/uc?id=1-21EnEXyzBeqzvpjKxzz2Q0GxacpJzGc

In [89]:
df = pd.read_csv("beer_df_large.csv", sep="\t")
df = df.drop(columns=['Name', 'Company', 'Region', 'ABV', 'Avg', 'Rate'])

In [90]:
df['Flat'] = df.apply(lambda x: '{}. {}'.format(x['Group'], x['Style']), axis=1)
df = df.drop(columns=['Style', 'Group'], axis=1)

In [91]:
df

,Review,Flat
0,From a 12oz bottle into a cocktail glass.\n\nC...,Bocks. Bock
1,"Great relaxing beer. Very mellow, great taste-...",Bocks. Bock
2,"Appearance: Clear, bright copper color. Frothy...",Bocks. Bock
3,As a German staying for holidays in the US I c...,Bocks. Bock
4,I would guess this is Shiner's #1 beer. It's p...,Bocks. Bock
...,...,...
174679,"A really well done, well balanced sour with an...",Wild/Sour Beers. Wild Ale
174680,"22oz bottle. Poured out a slightly hazy, brigh...",Wild/Sour Beers. Wild Ale
174681,"There's lots of apricot in this beer, just a t...",Wild/Sour Beers. Wild Ale
174682,Poured from a 22oz bomber into my Drie de Font...,Wild/Sour Beers. Wild Ale


In [92]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [93]:
labels = {}
for i, name in enumerate(df['Flat'].value_counts().index.tolist()):
    labels[name] = i

In [4]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = [labels[label] for label in df['Flat']]
        self.texts = [tokenizer(text, 
                                padding='max_length', 
                                max_length = 512, 
                                truncation=True, 
                                return_tensors="pt") 
                      for text in df['Review']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

In [95]:
from sklearn.model_selection import train_test_split

In [96]:
df_train, df_test = train_test_split(df, train_size=0.9, random_state=42, stratify=df['Flat'])
df_train, df_val = train_test_split(df_train, train_size=0.88889, random_state=42, stratify=df_train['Flat'])

print(len(df_train), len(df_val), len(df_test))

139746 17469 17469


In [5]:
class BertClassifier(nn.Module):
    def __init__(self, dropout=0.2):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 116)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

In [100]:
def train(model, train_data, val_data, learning_rate, epochs):
    print('Loading data into a dataset...')

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)
    print('Data loaded')

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):
            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [101]:
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading data into a dataset...
Data loaded


100%|██████████| 69873/69873 [1:16:57<00:00, 15.13it/s]


Epochs: 1 | Train Loss:  1.919                 | Train Accuracy:  0.195                 | Val Loss:  1.546                 | Val Accuracy:  0.315


100%|██████████| 69873/69873 [1:31:25<00:00, 12.74it/s]  


...

Epochs: 3 | Train Loss:  1.155                 | Train Accuracy:  0.431                 | Val Loss:  1.158                 | Val Accuracy:  0.425


100%|██████████| 69873/69873 [1:24:19<00:00, 13.81it/s]


Epochs: 4 | Train Loss:  1.031                 | Train Accuracy:  0.477                 | Val Loss:  1.118                 | Val Accuracy:  0.436


100%|██████████| 69873/69873 [1:24:33<00:00, 13.77it/s]


Epochs: 5 | Train Loss:  0.925                 | Train Accuracy:  0.523                 | Val Loss:  1.107                 | Val Accuracy:  0.441


In [102]:
torch.save(model, 'beert_flat.pt')

In [103]:
def evaluate(model, test_data):
    y_pred = []
    y_true = []
    print('Loading data...')
    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)
    print('Data loaded')

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    
    with torch.no_grad():
        for test_input, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            y_pred.append(output.argmax(dim=1))
            y_true.append(test_label)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    return y_pred, y_true

In [104]:
y_pred, y_true = evaluate(model, df_test)

Loading data...
Data loaded
Test Accuracy:  0.437


confusion matrix

In [32]:
t = [x.cpu() for x in y_true]

In [ ]:
classes = labels.keys()
cf_matrix = confusion_matrix([x.cpu() for x in y_true], [y.cpu() for y in y_pred])
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)
plt.savefig('output.png')